In [1]:
from keras.models import Sequential
from keras.layers import InputLayer, Conv2D, MaxPooling2D, Flatten, Dropout, Dense
from keras.layers.advanced_activations import LeakyReLU
from keras.utils import plot_model
from tensorflow import device
from castor import director

Using TensorFlow backend.


## Model

In [2]:
def new_model(name="Model"):
    """Model definition"""
    model = Sequential(name=name)
    model.add(InputLayer((62, 62, 1), name="Input0"))    
    model.add(Conv2D(16, (7,7), name="Conv1"))
    model.add(LeakyReLU(name="LeReLU1"))
    model.add(MaxPooling2D((3,3), padding="same", name="Max1"))   
    model.add(Conv2D(32, (5,5), name="Conv2"))
    model.add(LeakyReLU(name="LeReLU2"))
    model.add(MaxPooling2D((3,3), padding="same", name="Max2"))
    model.add(Flatten(name="Flat"))
    model.add(Dropout(0.2, name="Dropout"))    
    model.add(Dense(512))
    model.add(LeakyReLU(name="Leak")) #?
    model.add(Dense(192, activation="tanh"))
    model.add(Dense(96, activation="tanh"))
    model.add(Dense(3, activation="sigmoid"))
    return model

In [3]:
MODEL = new_model()
MODEL.summary()
plot_model(MODEL, "Model.png", True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Input0 (InputLayer)          (None, 62, 62, 1)         0         
_________________________________________________________________
Conv1 (Conv2D)               (None, 56, 56, 16)        800       
_________________________________________________________________
LeReLU1 (LeakyReLU)          (None, 56, 56, 16)        0         
_________________________________________________________________
Max1 (MaxPooling2D)          (None, 19, 19, 16)        0         
_________________________________________________________________
Conv2 (Conv2D)               (None, 15, 15, 32)        12832     
_________________________________________________________________
LeReLU2 (LeakyReLU)          (None, 15, 15, 32)        0         
_________________________________________________________________
Max2 (MaxPooling2D)          (None, 5, 5, 32)          0         
__________

In [4]:
MODEL.compile(optimizer="sgd", loss="mse", metrics=['accuracy'])

In [5]:
with device('/cpu:0'):
    MODEL.fit_generator(director("lamost"), steps_per_epoch=128, epochs=1,
                        validation_data=director("lamost"), validation_steps=48)

Epoch 1/1
128/128 [==============================] - 1s - loss: 0.0299 - acc: 0.7500 - val_loss: 0.0233 - val_acc: 0.7500


In [6]:
MODEL.save("lamost.h5")